<a href="https://colab.research.google.com/github/samyff12/LATAM/blob/main/IMAE_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Actualizacion IMAE LATAM**

### Paquetes y meses

In [1]:
from bs4 import BeautifulSoup
import pandas as pd    # ! pip install --upgrade xlrd  por si no se actualiza pandas
import numpy as np
import requests
import urllib.request as urllib2
import re
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
!pip install XlsxWriter
from google.colab import files
import datetime
import time
import warnings
from pandas.core.common import SettingWithCopyWarning # para el warning especifico

#!pip install kora -q
#from kora.selenium import wd

# Selenium
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
# Argumentos de Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
# En caso de contar con archivos en formato pdf: 
# !pip install pymupdf            
# !pip install tabula-py
# import tabula as tb
# import fitz  # this is pymupdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 5.2 MB/s 
     |████████████████████████████████| 358 kB 38.2 MB/s 
     |████████████████████████████████| 138 kB 17.5 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 4.0 MB 35.7 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, bu

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu b

In [2]:
! pip install --upgrade xlrd
! pip install --upgrade openpyxl
##html = requests.get('http://www.secmca.org/chart/?parent=Producci%C3%B3n&scid=2&cid=1&scsid=0&son=%C3%8Dndice%20Mensual%20de%20la%20Actividad%20Econ%C3%B3mica&url=11/14/3-13-50-17-4-52-161/39/265/19-258/444/' +
 #                   '20211-20212-20213-20214-20215-20216-20217-20218-20219-202110-202111-202112-20221-20222-20223-20224-20225-20226-20227-20228-20229-202210-202211-202212' +
 #                   '-&all_vars=2|IMAE,%20variaci%C3%B3n%20interanual')
#soup = BeautifulSoup(html.text,'lxml')
#table = soup.find_all('p', class_='text-right')
#tablec = soup.find_all('th', colspan='1')

#indicadores=[]
#paises =[]
#for i in range(len(table)):
 # ind = float(table[i].text)
#  pais = tablec[i].text
#  indicadores.append(ind)
 # paises.append(pais)

#imae_latam =  pd.DataFrame(indicadores,index = paises,columns = ['IMAE'])
#print(f'IMAE a {fecha}')
#imae_latam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 1.7 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
meses = [0,[1,'Enero'],[2,'Febrero'],[3,'Marzo'],[4,'Abril'],[5,'Mayo'],[6,'Junio'],[7,'Julio'],[8,'Agosto'],
         [9,'Septiembre'],[10,'Octubre'],[11,'Noviembre'],[12,'Diciembre']]
mes3 = {'ene':'Enero','feb':'Febrero','mar':'Marzo','abr':'Abril','may':'Mayo','jun':'Junio','jul':'Julio',
        'ago':'Agosto','sep':'Septiembre','oct':'Octubre','nov':'Noviembre','dic':'Diciembre'}
meses_dict = {0:'none',1:'Enero',2:'Febrero',3:'Marzo',4:'Abril',5:'Mayo',6:'Junio',7:'Julio',8:'Agosto',
         9:'Septiembre',10:'Octubre',11:'Noviembre',12:'Diciembre'}

header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

#Warnings

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Para obtener las series de todo el año 2021-2022


### **SECMCA**

#### Publicaciones disponibles las fechas: 

#####    **El Salvador**
- 29 Abril


#####    **Panama**
- 21 Mar (se publica Dic y enero )
- 19 Abr
- 19 May
- 20 Jun
- 19 Jul
- 18 ago


#####    **Honduras**
-. Preliminar 20 de cada mes, con 50 días de rezago y
revisado 30 días después

In [4]:
html2 = requests.get('https://www.secmca.org/chart/?parent=Producci%C3%B3n&scid=2&cid=1&scsid=0&son=%C3%8Dndice%20Mensual%20de%20la%20Actividad%20Econ%C3%B3mica&url=11/14/3-13-50-17-4-52-161/39/265/19-258/81/20171-20172-20173-20174-20175-20176-20177-20178-20179-201710-201711-201712-20181-20182-20183-20184-20185-20186-20187-20188-20189-201810-201811-201812-20191-20192-20193-20194-20195-20196-20197-20198-20199-201910-201911-201912-' +
'20201-20202-20203-20204-20205-20206-20207-20208-20209-202010-202011-202012-20211-20212-20213-20214-20215-20216-20217-20218-20219-202110-202111-202112-&all_vars=1|IMAE').text

In [5]:
# INDICE VERSION
html1 = requests.get('http://www.secmca.org/chart/?parent=Producci%C3%B3n&scid=2&cid=1&scsid=0&son=%C3%8Dndice%20Mensual%20de%20la%20Actividad%20Econ%C3%B3mica&url=11/14/3-13-50-17-4-52-161/39/265/19-258/81/' +
  '20171-20172-20173-20174-20175-20176-20177-20178-20179-201710-201711-201712-20181-20182-20183-20184-20185-20186-20187-20188-20189-201810-201811-201812-20191-20192-20193-20194-20195-20196-20197-20198-20199-201910-201911-201912-' +
  '20201-20202-20203-20204-20205-20206-20207-20208-20209-202010-202011-202012-20211-20212-20213-20214-20215-20216-20217-20218-20219-202110-202111-202112-20221-20222-20223-20224-20225-20226-20227-20228-20229-202210-202211-202212' +
  '-&all_vars=1|IMAE').text
soup1 = BeautifulSoup(html1,'lxml')

tabpais = soup1.find_all('th', colspan='1')
table = soup1.find_all('p', class_='text-left')


In [6]:
fechas = []
for i in range(len(table)):
  x = re.sub(" ","",table[i].text)
  fechas.append(x.replace("\n","").replace('Setiembre','Septiembre'))

paises = [tabpais[i].text for i in range(len(tabpais))]
fechas[-1]

'2022-Abril'

In [7]:
#INDICE
table1 = soup1.find_all('p', class_='text-right')
matriz = np.array([])
for i in table1:
  matriz = np.array(np.append(matriz,i))
matriz = np.where(matriz=="--", np.nan, matriz).astype(float)
matriz = matriz.reshape(-1,7) # 7 paises
imaeLA = pd.DataFrame(matriz,index=fechas,columns=paises)
imaeLA.tail()

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá
2021-Diciembre,120.64,123.45,140.75,279.03,180.08,212.34,396.97
2022-Enero,106.97,108.41,134.93,248.73,168.58,186.90,341.51
2022-Febrero,108.55,109.35,134.59,248.74,154.54,196.08,365.41
2022-Marzo,118.00,119.08,139.63,267.13,158.90,195.02,400.45
2022-Abril,107.79,NaN,135.90,250.38,NaN,193.18,NaN


In [8]:
# PARA SOLO ACTUALIZAR SECMCA dentro de imaelatam

# for col in imaeLA.columns:
#   imaela[col] = imaeLA[col]

# imaela.tail(3)

import re
tabpais = soup.find_all('th', colspan='1')
table = soup.find_all('p', class_='text-left')
fechas = []
paises =[]
for i in range(len(table)):
  x = re.sub(" ","",table[i].text)
  fechas.append(x.replace("\n",""))
  pais = tabpais[i].text
  #indicadores.append(ind)
  paises.append(pais)
fechas[-1]

table1 = soup.find_all('p', class_='text-right')
matriz = np.array([])
for i in table1:
  matriz = np.array(np.append(matriz,i))
matriz = np.where(matriz=="--", np.nan, matriz).astype(np.float)
matriz = matriz.reshape(-1,7) # 7 paises
imaeLA = pd.DataFrame(matriz,index=fechas,columns=paises)
imaeLA

### **Argentina**


In [9]:
url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_emae_mensual_base2004.xls"
urllib2.urlretrieve(url, "test.xls")
data = pd.read_excel('test.xls',header=2)[2:].reset_index(drop=True)

data.iloc[215,0] = 2022 # no añadieron 2022 en el excel

data1 = data.iloc[:,:4].dropna(subset = [data.columns[1]]) # data.columns[1] es la columna de los meses
data1.reset_index(drop=True,inplace=True)
data1[data1.columns[0]] = data1[data1.columns[0]].ffill() # forward fill # para los años   data1.columns[0] es la columna de los años
data1 = data1.iloc[11:,:3]

In [10]:
data1['fecha'] = data1[data1.columns[0]].astype(int).astype(str) +"-"+ data1[data1.columns[1]]
data1.set_index(data1.fecha,inplace=True)
data1.iloc[:,2]

fecha
2005-Enero         98.473360
2005-Febrero       96.118866
2005-Marzo        110.656426
2005-Abril        116.545190
2005-Mayo         126.661943
                     ...    
2021-Noviembre    145.254982
2021-Diciembre    145.823877
2022-Enero        136.199452
2022-Febrero      137.408300
2022-Marzo        153.350828
Name: Índice Serie Original\n2004=100, Length: 207, dtype: float64

In [11]:
data1.index

Index(['2005-Enero', '2005-Febrero', '2005-Marzo', '2005-Abril', '2005-Mayo',
       '2005-Junio', '2005-Julio', '2005-Agosto', '2005-Septiembre',
       '2005-Octubre',
       ...
       '2021-Junio', '2021-Julio', '2021-Agosto', '2021-Septiembre',
       '2021-Octubre', '2021-Noviembre', '2021-Diciembre', '2022-Enero',
       '2022-Febrero', '2022-Marzo'],
      dtype='object', name='fecha', length=207)

In [12]:
imaela = imaeLA.merge(data1.iloc[:,2],how='left',left_index=True,right_index=True)
imaela.columns = [*imaela.columns[:-1], 'Argentina']
imaela.tail(2)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina
2022-Marzo,118.00,119.08,139.63,267.13,158.9,195.02,400.45,153.350828
2022-Abril,107.79,NaN,135.90,250.38,NaN,193.18,NaN,NaN


### **Guatemala**

In [13]:
guat_url = 'https://www.banguat.gob.gt/es/page/indice-mensual-de-la-actividad-economica-imae-ano-de-referencia-2013'

req = urllib2.Request(url=guat_url, headers=header) 
page = urllib2.urlopen(req).read()

soupguat = BeautifulSoup(page)
guat_links = [a['href'] for a in soupguat.select("a[href*=.xlsx]")]
guat_links

['/sites/default/files/banguat/Publica/IMAE/2013/Cuadros_y_graficas_IMAE_abr2022.xlsx',
 '/sites/default/files/banguat/publica/IMAE/2013/Empalme_IMAE_abr_2022.xlsx']

In [14]:
time.sleep(np.random.randint(0,4,1) + np.random.random()) # esperar un tiempo aleatorio para fines
urllib2.urlretrieve('https://www.banguat.gob.gt' + guat_links[0], "guat.xlsx")
df_guat = pd.read_excel('guat.xlsx', sheet_name=1, header=7)

In [15]:
guat = df_guat.iloc[:,:2].dropna(how='any', axis=0)
guat.columns = ['fecha', 'Guatemala']
guat['fechas'] = pd.to_datetime(guat.fecha).dt.year.astype(str) + '-' + pd.to_datetime(guat.fecha).dt.month.map(meses_dict).astype(str)
guat.set_index('fechas', inplace=True)
guat.tail(2)

,fecha,Guatemala
fechas,,
2022-Marzo,2022-03-01 00:00:00,139.630203
2022-Abril,2022-04-01 00:00:00,135.895941


In [16]:
for index in imaela.index:
  try:
    imaela.loc[index]['Guatemala'] = guat.loc[index]['Guatemala']

  except KeyError as e:
    print(f'Aun no sale {e}')

In [17]:
imaela.tail(2)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina
2022-Marzo,118.00,119.08,139.630203,267.13,158.9,195.02,400.45,153.350828
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN


### **Nicaragua**

In [18]:
nic_url = 'https://www.bcn.gob.ni/sites/default/files/estadisticas/sector_real/produccion/Cuadros_de_salida_IMAE.xlsx'
urllib2.urlretrieve(nic_url, 'nic.xlsx')
df_nic = pd.read_excel('nic.xlsx', sheet_name=0)

In [77]:
nic = df_nic.iloc[-197:,1:4] # Seleccionar desde 2006 y hasta la columna de indice
nic.columns = ['Año', 'Mes', 'Nicaragua']

#Borrar notas al margen del excel
nic.Nicaragua.fillna('borrar', inplace=True)
nic = nic[nic.Nicaragua != 'borrar']
nic['Nicaragua'] = pd.to_numeric(nic.Nicaragua, errors='coerce')
nic['Año']

# Set index con fecha
nic['Fecha'] = nic.Año.ffill().astype(str) + '-' + nic.Mes.str.lower().map(mes3)
nic.set_index('Fecha', drop=True, inplace=True)

nic.tail(2)

,Año,Mes,Nicaragua
Fecha,,,
2022.0-Febrero,NaN,Feb,154.541556
2022.0-Marzo,NaN,Mar,158.898031


In [92]:
nic['Año'].apply(lambda x: int(x) if type(x)==int)

SyntaxError: ignored

In [ ]:
for index in imaela.index:
  try:
    imaela.loc[index]['Nicaragua'] = nic.loc[index]['Nicaragua']

  except KeyError as e:
    print(f'Aun no sale {e}')
    

In [21]:
imaela.tail(2)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina
2022-Marzo,118.00,119.08,139.630203,267.13,158.9,195.02,400.45,153.350828
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN


### **Chile**

#### Publicaciones disponibles las fechas:
 
-   1ero o 2do día de cada mes

In [22]:
url = "https://si3.bcentral.cl/estadisticas/Principal1/excel/CCNN/IMACEC/Excel/IMACEC.xlsx"
urllib2.urlretrieve(url, "chile.xlsx")
df = pd.read_excel('chile.xlsx',header=5,keep_default_na=False)

In [23]:
df.columns = df.columns.str.strip() # eliminar espacios innecesarios
chile = df.loc[1:,[df.columns[0],df.columns[1],'Imacec']][:-10] # excluimos las ultimas 10 filas (pie de pagina)
chile.rename(columns = {'Imacec':'Chile'}, inplace = True) # renombramos
chile['Chile1'] = pd.to_numeric(chile.Chile).pct_change(12)*100
###
chile[chile.columns[0]] = pd.to_numeric(chile[chile.columns[0]], errors='coerce').ffill().bfill().astype(int)
 # las columnas del excel contienen string en blanco por lo que se transformaron a numero antes de hacer el forward fill
 # el backward fill es para asegurar que no quede NaN por detras y dificulte la creacion del indice
###

chile['mes'] = chile[chile.columns[1]].map(mes3).bfill() # creando mes y usando el bfill por el mismo motivo explicado arriba

#Creamos el indice
chile.index = chile[chile.columns[0]].astype('str') +"-"+ chile.mes

chile.tail(2)

,,Unnamed: 1,Chile,Chile1,mes
2022-Marzo,2022,mar,114.688751,6.375828,Marzo
2022-Abril,2022,abr,109.32661,6.937841,Abril


In [24]:
# df.head()
# df.columns = df.columns.str.strip()

In [25]:
 pd.to_numeric(chile[chile.columns[0]], errors='coerce')
 chile = chile.iloc[5:] # para contar desde 2018

In [26]:
# right para contar con la ultima informacion de chile, ya que publica primero
imaela = imaela.merge(chile['Chile'],how='left',left_index=True,right_index=True)
#imaelatam = imaelatam[~imaelatam.index.duplicated(keep='last')].reindex(imaela.index) # hay duplicados, los eliminamos y reindexamos

In [27]:
imaela.tail(5)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile
2021-Diciembre,120.64,123.45,140.752165,279.03,180.08,212.34,396.97,145.823877,120.318627
2022-Enero,106.97,108.41,134.932148,248.73,168.58,186.90,341.51,136.199452,104.597142
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.63122
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN,109.32661


### **Brasil**

In [28]:
urljson = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.24363/dados?formato=json"
urllib2.urlretrieve(urljson, "brasil.json")
bra = pd.read_json('brasil.json')
bra.rename(columns={'valor':'Brasil'},inplace=True)

bra['Brasil1'] = bra.Brasil.pct_change(12)*100 # calculo tasa de crecimiento
bra['date'] = pd.to_datetime(bra['data'],format='%d/%m/%Y') # transformamos a formato fecha
# Creamos indice
bra['fecha'] = pd.DatetimeIndex(bra['date']).year.astype(str) +"-"+pd.DatetimeIndex(bra['date']).month.astype(str)

nombre_mes = [meses[i][1] for i in pd.DatetimeIndex(bra['date']).month.tolist()] # obtenemos nombre del mes
bra['fecha'] = pd.DatetimeIndex(bra['date']).year.astype(str) +"-"+ nombre_mes # creamos indice
bra.set_index(bra.fecha,inplace=True) #reemplazamos indice


bra.tail(1)

,data,Brasil,Brasil1,date,fecha
fecha,,,,,
2022-Febrero,01/02/2022,135.52,0.661071,2022-02-01,2022-Febrero


In [29]:
imaela = pd.merge(imaela,bra['Brasil'],how='left',left_index=True,right_index=True) # combinamos la data

In [30]:
#    imaela

### **Mexico**

In [31]:
url = "https://www.inegi.org.mx/contenidos/temas/economia/cn/igae/tabulados/ori/IGAE_1.xlsx"
urllib2.urlretrieve(url, "test1.xlsx")
mex = pd.read_excel('test1.xlsx',header=3,nrows=3)
mex = mex.T.reset_index(drop=True) # transponemos y estandarizamos el indice
mex.drop(0,inplace=True) # borramos primera fila
mex[2] = round(mex[2].astype(float),2) # leyendo tasas de crecimiento en el formato numerico y redondeando
mex[0] = mex[0].ffill()

In [32]:
mex[1] = mex[1].replace(['oR','oP','lR','lP','eR','eP',' '],['o','o','l','l','e','e',''], regex=True) #limpiar superindices "Revisado" "Preliminar"
mex[0] = mex[0].replace(['P','R',' '],['','',''],regex=True) # limpiar preliminar de fecha
mex.index = mex[0].astype('str') +"-"+ mex[1] #creamos indice para base de datos
mex.columns = [*mex.columns[:-1], 'México'] # nombramos columna
mex.dropna().tail(2) # mostrar ultimo valor no nulo

,0,1,México
2022-Febrero,2022,Febrero,105.28
2022-Marzo,2022,Marzo,111.96


In [33]:
imaela = pd.merge(imaela,mex['México'],how='left',left_index=True,right_index=True) # combinamos las series

In [34]:
imaela.tail(3)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile,Brasil,México
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.63122,135.52,105.28
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751,NaN,111.96
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN,109.32661,NaN,NaN


### **Perú**
- Falta fecha

In [35]:
# url = 'https://www.inei.gob.pe/media/principales_indicadores/CalculoPBI_104.zip' #cambio de 103 a 104 (feb mar)

In [36]:
urlpe= 'https://www.inei.gob.pe'
wd.get(urlpe)
souppe = BeautifulSoup(wd.page_source,'lxml')
link_html = souppe.find_all('a',class_='btninei',title="documentos adicionales") # para encontrar el nombre del link de indice peru
for a in link_html:
  link = a['href']
  print(link)

urlpe = 'https://www.inei.gob.pe/' + link

/media/principales_indicadores/CalculoPBI_106.zip


In [37]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
zip, _ = urllib2.urlretrieve(urlpe)
with zipfile.ZipFile(zip, "r") as f:
    f.extractall('/content')

In [38]:
indicador = [f.namelist()[i] for i in range(len(f.namelist())) if 'Indicador Mensual' in f.namelist()[i]][0] # el 0 es para que elija el string de la lista
df = pd.read_excel(indicador,sheet_name=1,header=3)[1:] # el indice esta en la segunda hoja del excel

In [39]:
indicador

'Indicador Mensual de la Producción Nacional 04 2022 r.xlsx'

In [40]:
df.dropna(how='any', inplace=True)
pe = df.filter(like='Índice Global') # df.filter(like='Índice Global')| regex='spike|spke' se puede usar regex para en caso de que escriban las columnas mal
pe['YM'] = df['Año y Mes'].astype(int).astype(str)

#  IG.1 Mensual   IG.2 Acumulado  IG.3 Anual
pe['fecha'] =  pe['YM'].str[0:4] +'-'+ pe['YM'].str[4:7].astype(int).map(meses_dict)  # fecha
pe.set_index('fecha',inplace=True)

pe.rename(columns={pe.columns[0]:'Perú',pe.columns[1]:'Perú_i',pe.columns[2]:'Perú_a'},inplace=True)

In [41]:
imaela = imaela.merge(pe['Perú'],how='left',left_index=True,right_index=True)

In [42]:
imaela.tail(3)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile,Brasil,México,Perú
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.63122,135.52,105.28,163.628001
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751,NaN,111.96,171.874742
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN,109.32661,NaN,NaN,171.427214


### **Ecuador**

In [43]:
url = "https://contenido.bce.fin.ec/documentos/PublicacionesNotas/Catalogo/IEMensual/Adelantos/IEM-452-e.xlsx"
urllib2.urlretrieve(url, "Ecuador.xlsx")
df = pd.read_excel('Ecuador.xlsx',header=7,keep_default_na=False)[1:13].reset_index()

ecu = (df.set_index(['Unnamed: 0','index'])
.stack()
.reset_index()
.rename(columns={'Unnamed: 0':'mes','level_2':'Year'})).sort_values(['Year','index']) # ingenieria de base de datos

# Elaboracion de indice
ecu['Mes'] = ecu.mes.str.lower().map(mes3)
ecu.insert(3, 'fecha', ecu.Year.astype(str) + '-' + ecu.Mes)
ecu.index = ecu.fecha

# Variacion interanual
ecu.rename(columns={0:'Ecuador'},inplace=True)
ecu['Ecuador1'] = pd.to_numeric(ecu['Ecuador'],errors='coerce').pct_change(12)*100

ecu.Ecuador.tail(12)

fecha
2022-Enero         167.160031
2022-Febrero       151.053282
2022-Marzo         167.054122
2022-Abril         165.453821
2022-Mayo                    
2022-Junio                   
2022-Julio                   
2022-Agosto                  
2022-Septiembre              
2022-Octubre                 
2022-Noviembre               
2022-Diciembre               
Name: Ecuador, dtype: object

In [44]:
imaela = pd.merge(imaela,ecu['Ecuador'],how='left',left_index=True,right_index=True)

In [45]:
imaela.tail(3)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile,Brasil,México,Perú,Ecuador
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.63122,135.52,105.28,163.628001,151.053282
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751,NaN,111.96,171.874742,167.054122
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN,109.32661,NaN,NaN,171.427214,165.453821


df = np.array(tb.read_pdf('ecuador.pdf',pages = '5',stream=True,multiple_tables=True,pandas_options={'header':None}))
dff = pd.DataFrame(df.reshape(-1,5)) # en 2022 el reshape debe ser (-1,6)

dff[dff.iloc[3,2].split(' ')] = dff[2].str.split(' ',expand=True)
if dff.iloc[3,3] in range(2020,2040,1):
  dff[int(dff.iloc[3,3])] = dff[3]
  dff[int(dff.iloc[3,4])] = dff[4]
else:
  dff[int(dff.iloc[3,4])] = dff[4]

if dff.iloc[18,0] in ['Ene']:
    ecu = dff.iloc[18:30,[0,-3,-2,-1]].reset_index(drop=True)
else:
    ecu = dff.iloc[19:31,[0,-3,-2,-1]].reset_index(drop=True)
 
ecu['interanualAnterior'] = ecu.iloc[:,-2].astype(float)/ecu.iloc[:,-3].astype(float)*100-100
ecu['Ecuador'] = ecu.iloc[:,-2].astype(float)/ecu.iloc[:,-3].astype(float)*100-100

listames = [meses[1:][i][1] for i in range(len(meses[1:]))]
ecu['mes'] = listames

ecu.index = str(ecu.columns[-4]) + '-' + ecu.mes
ecu.dropna().tail(1)

### **Bolivia**
- Falta fecha

In [46]:
urlbol = 'https://www.ine.gob.bo/index.php/estadisticas-economicas/indice-global-de-actividad-economica-igae/#1584545313102-06cf787f-a790'

req = urllib2.Request(url=urlbol, headers=header) 
page = urllib2.urlopen(req).read()

In [47]:
soupbol = BeautifulSoup(page,'lxml')
links = [a['href'] for a in soupbol.select("a[href*=nube.ine.gob.bo/index.php]")] # extraemos links que contengan nube.ine.gob
 # ultimos link disponibles
links # IGAE por actividad economica es el segundo

['https://nube.ine.gob.bo/index.php/s/lhzNu4ISQEuXS1M/download',
 'https://nube.ine.gob.bo/index.php/s/6Szm6RmYSBeRnsD/download',
 'https://nube.ine.gob.bo/index.php/s/qNQexpgoZroGCYG/download',
 'https://nube.ine.gob.bo/index.php/s/vsfGqiIoEXGddwJ/download',
 'https://nube.ine.gob.bo/index.php/s/k0bRzzNhBXb7Ae2/download']

In [48]:
urllib2.urlretrieve(links[1], "Bolivia.xlsx")
#https://nube.ine.gob.bo/index.php/s/6Szm6RmYSBeRnsD/download

('Bolivia.xlsx', <http.client.HTTPMessage at 0x7f389ae88d50>)

In [49]:
df = pd.read_excel('Bolivia.xlsx',header=9)

In [50]:
df.set_index(df.iloc[:,0],inplace=True)
df = df.iloc[:,1:] # eliminar primera columna y ultimas 3 filas
df = df.T.reset_index()

In [51]:
df.tail(1)

ACTIVIDAD ECONÓMICA,index,0,INDICE GENERAL,"AGRICULTURA, PECUARIO, SILVICULTURA, CAZA Y PESCA",PETRÓLEO CRUDO Y GAS NATURAL,MINERALES METÁLICOS Y NO METÁLICOS,INDUSTRIA MANUFACTURERA,"- Alimentos, Bebidas y Tabaco",- Otras Industrias,"ELECTRICIDAD, GAS Y AGUA",...,- Servicios Financieros,- Servicios a las Empresas,- Propiedad de Vivienda,SERVICIOS DE LA ADMINISTRACIÓN PÚBLICA,OTROS SERVICIOS(1),Servicios Bancarios Imputados,"Derechos s/Importaciones, IVAnd, IT y otros Imp. Indirectos",Fuente: INSTITUTO NACIONAL DE ESTADÍSTICA,(p): Preliminar,"(1): Incluye las actividades de Restaurantes y Hoteles asi como Servicios Comunales, Sociales, Personales y Domestico"
167,Unnamed: 168,Dic,340.382222,181.412225,282.869492,213.955691,336.039536,394.347375,287.56732,561.24669,...,1186.602013,358.301454,214.677743,414.025102,259.064724,1022.419201,554.709631,NaN,NaN,NaN


In [52]:
# PARA EXTRAER AÑO DE COLUMNA INDEX
df['index'] = df['index'].astype(str).str.replace('\(p\)','',regex=True)
df['index'] = df['index'].str.replace('^0.[0-9]','nan',regex=True)      # reemplaza los numeros decimales regex que empiece por 0. y le siga cualquier num
df['index'] = df['index'].str.replace('^0$','nan',regex=True)    # reemplaza toda fila con 0 como unico digito
df['index'] = pd.to_numeric(df['index'],errors='coerce').ffill().astype(int)

In [53]:
df['fecha'] = df['index'].astype(str) +'-'+ df[0].str.lower().map(mes3) # columna fecha
df.set_index('fecha',inplace=True)
df.rename(columns={df.columns[2]:'Bolivia'},inplace=True)

In [54]:
imaela = imaela.merge(df['Bolivia'],how='left',left_index=True,right_index=True)

In [55]:
imaela.tail(6)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile,Brasil,México,Perú,Ecuador,Bolivia
2021-Noviembre,117.32,116.92,135.343372,264.88,167.49,204.87,333.33,145.254982,116.334812,140.16,112.97,177.301379,165.0701,333.675632
2021-Diciembre,120.64,123.45,140.752165,279.03,180.08,212.34,396.97,145.823877,120.318627,142.07,112.33,195.058393,176.012995,340.382222
2022-Enero,106.97,108.41,134.932148,248.73,168.58,186.90,341.51,136.199452,104.597142,132.27,107.32,166.075957,167.160031,NaN
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.63122,135.52,105.28,163.628001,151.053282,NaN
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751,NaN,111.96,171.874742,167.054122,NaN
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN,109.32661,NaN,NaN,171.427214,165.453821,NaN


### **El Salvador**

In [56]:
from selenium import webdriver
pagina = 'https://www.bcr.gob.sv/bcrsite/?cdr=174&lang=es'

In [57]:
##### **Otra forma de selenium**
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wdsv = webdriver.Chrome('chromedriver',options=chrome_options)
wdsv.get(pagina)

In [58]:
 #BC El Salvador

# Clicks pertinentes para llegar a la tabla con indices imae SV
wdsv.find_element(By.CLASS_NAME,'linkBtnNaranja').click() # ejecutar consulta

In [59]:
# PARA EXTRAER LOS VALORES DE LOS INDICES

tr = wdsv.find_element(By.ID,'tbl_setValores')
data = tr.find_element(By.TAG_NAME,'tr')
indices_salvador = pd.to_numeric(data.text.split(' ')) 


# PARA EXTRAER LAS FECHAS PARA EL MERGE
soupsv = BeautifulSoup(wdsv.page_source,'html.parser')
anios_sv = soupsv.find_all('td', style='text-align:center')
mes_sv = soupsv.find_all('center')

fecha1 = [anios_sv[i].text for i in range(len(anios_sv))] # lista con años 
fecha2 = [mes_sv[i].text.replace(' ','')[:3] for i in range(1,len(mes_sv))] # lista con meses en formato mmm

fecha=[]
for i in fecha1:
  for j in range(1,13):
    fecha.append(i + "-" + meses_dict[j])

In [60]:
# CREAR DF
elsv = pd.DataFrame(index = fecha,columns=['sv'])
for i in range(len(indices_salvador)):
  elsv.sv[i] = indices_salvador[i]

imaela['El Salvador'] = elsv.loc[imaela.index] # reemplazar valor en la tabla imaela

elsv.loc[imaela.index][-3:]

,sv
2022-Febrero,109.35
2022-Marzo,119.08
2022-Abril,NaN


### **Paraguay**

- Falta fecha

In [61]:
url = 'https://www.bcp.gov.py/indicador-mensual-de-actividad-economica-del-paraguay-imaep-i471'
urlpy = 'https://www.bcp.gov.py/anexo-estadistico-del-informe-economico-i365'
#the URL where you are requesting at
ssl._create_default_https_context = ssl._create_unverified_context
req = urllib2.Request(url=urlpy, headers=header) 
page = urllib2.urlopen(req).read()

In [62]:
wdpy = webdriver.Chrome('chromedriver',options=chrome_options)
wdpy.get(urlpy)

In [63]:
souppy = BeautifulSoup(wdpy.page_source,'lxml')
links = [a['href'] for a in souppy.select("a[href*=.xlsx]")] # extraemos links que contengan .xlsx
 # ultimos link disponibles
links

['userfiles/files/Anexo_Estad%C3%ADstico_del_Informe_Econ%C3%B3mico_16_06_2022.xlsx']

In [64]:
ssl._create_default_https_context = ssl._create_unverified_context
url2py = 'https://www.bcp.gov.py/' + links[0]
req2 = requests.get(url2py, headers = header, verify=False)  # nueva forma de descargar excel file de pagina de py
#urllib2.urlretrieve(url2py, "py.xlsx")

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  method,


In [65]:
import datetime
dfpy = pd.read_excel(req2.content,sheet_name='CUADRO 9',header=9).iloc[:-3,1:] # con limpieza iloc CUADRO 9 es IMAEP
dfpy.rename(columns={dfpy.columns[1]:"Paraguay"},inplace=True)

In [66]:
# FECHA COLUMNA
dfpy['A'] = pd.DatetimeIndex(dfpy[dfpy.columns[0]]).year
dfpy['M'] = pd.DatetimeIndex(dfpy[dfpy.columns[0]]).month.map(meses_dict)
dfpy['fecha'] = dfpy.A.astype(str) + "-" + dfpy.M.astype(str)
dfpy.set_index('fecha',inplace=True)
dfpy['Paraguay'].tail()

fecha
2021-Diciembre    140.144044
2022-Enero        114.314550
2022-Febrero      115.403306
2022-Marzo        123.845187
2022-Abril        110.521665
Name: Paraguay, dtype: float64

In [67]:
imaela = imaela.merge(dfpy['Paraguay'],how='left',left_index=True,right_index=True)

In [68]:
imaela.tail(3)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile,Brasil,México,Perú,Ecuador,Bolivia,Paraguay
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.63122,135.52,105.28,163.628001,151.053282,NaN,115.403306
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751,NaN,111.96,171.874742,167.054122,NaN,123.845187
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN,109.32661,NaN,NaN,171.427214,165.453821,NaN,110.521665



url2 = 'https://www.bcp.gov.py/' + links[0] # -(numero mes)
opener = urllib2.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib2.install_opener(opener)
urllib2.urlretrieve(url2, "py.pdf")




'With pdf file

with fitz.open("py.pdf") as doc:
    text = ""
    for page in doc:
        text += page.get_text()

fechapy = text.split(' |',2)[1] # con el espacio
fechapy = fechapy.split(' ')[1] +'-'+fechapy.split(' ')[0]

Debido a que las tasas son iguales en Enero

if 'Enero' not in fechapy:
  valorpyinteranual = re.findall(r'[0-9]+,[0-9]+',text)[0] # para obtener tasa interanual          el regex sirve para buscar los registros numericos separados por coma
  valorpyacum = re.findall(r'[0-9]+,[0-9]+',text)[1] # para obtener tasa acumulada interanual
  valorpyinteranual = float(valorpyinteranual.replace(',','.'))
  valorpyacum = float(valorpyacum.replace(',','.'))
else:
   valorpyinteranual = re.findall(r'[0-9]+,[0-9]+',text)[0]
   valorpyinteranual = float(valorpyinteranual.replace(',','.'))
   valorpyacum = valorpyinteranual

print(fechapy)
print(valorpyinteranual,' Tasa interanual')
print(valorpyacum, ' Tasa acumulada interanual')


py = pd.DataFrame({'Fecha':['2021-Enero','2021-Febrero','2021-Marzo','2021-Abril','2021-Mayo','2021-Junio','2021-Julio','2021-Agosto','2021-Septiembre','2021-Octubre','2021-Noviembre','2021-Diciembre'],
                   'Paraguay_a':[-5.6,-3.0,0.7,4.7,6.3,6.3,6.0,5.7,0,0,0,0],
                   'Paraguay_i':[-5.6,-0.5,8.0,19.3,12.4,6.6,3.0,4.2,0,0,0,0]})
py.index = py.Fecha
dic = {'Fecha':fechapy,'Paraguay_a':valorpyacum,'Paraguay_i':valorpyinteranual}
py.loc[fechapy] = [fechapy,valorpyacum,valorpyinteranual]
py

In [69]:
imaela['a'] = [imaela.index.str.split('-')[i][0] for i in range(len(imaela.index))]
imaela = imaela.apply(pd.to_numeric)

In [70]:
# Crecimiento interanual
imaela1 = imaela.pct_change(12,fill_method=None)*100
imaela1.dropna(how='all',inplace=True) # NAs drop
imaela1.drop('a',axis=1,inplace=True)

In [71]:
# Crecimiento acumulado interanual
imaela2 = imaela.groupby('a').cumsum().pct_change(12,fill_method=None)*100
imaela2.dropna(how='all',inplace=True) # NAs drop

In [72]:
#!pip install selenium
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#import sys
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#from selenium import webdriver
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')
#wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

### **Colombia**

In [ ]:
!pip install fake_useragent
from fake_useragent import UserAgent
ua = UserAgent()
print(ua.chrome)
print(ua.random)
url1 = 'https://www.dane.gov.co/index.php/estadisticas-por-tema/cuentas-nacionales/indicador-de-seguimiento-a-la-economia-ise'

header_col = {'authority': 'www.dane.gov.co', 'method': 'GET', 'path': '/index.php/estadisticas-por-tema/cuentas-nacionales/indicador-de-seguimiento-a-la-economia-ise',
'scheme': 'https', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'referer':'https://www.dane.gov.co/index.php/estadisticas-por-tema/cuentas-nacionales/indicador-de-seguimiento-a-la-economia-ise',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'es-ES,es;q=0.9',
'cache-control': 'max-age=0',
'cookie': '_ga=GA1.3.1467282597.1625839173; _ga_T3L41X6043=GS1.1.1632235276.4.0.1632235284.0; ec1d9c9ecb5c2f4ef7998e3e52b19fdf=bvqfvrh7p0e0kgp1vf8bg29s68; _gid=GA1.3.2105323501.1637672614',
'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "Windows",
'sec-fetch-dest': 'document',
'sec-fetch-mode': 'navigate',
'sec-fetch-site': 'none',
'sec-fetch-user': '?1',
'upgrade-insecure-requests':' 1',
'user-agent': str(ua.random)}

wd.get(url1) #leer la pagina

soupco = BeautifulSoup(wd.page_source,'lxml')
links = [a['href'] for a in soupco.select("a[href*=Anex_ISE]")] # extraemos links que contengan 21.xlsx 
 #### ultimos link disponibles
links

url2 = 'https://www.dane.gov.co' + links[0] # si esta el nuevo mes restarle 1 al indice de links
time.sleep(np.random.randint(0,5,1) + np.random.random())
urllib2.urlretrieve(url2, "colombia.xlsx")


df = pd.read_excel('colombia.xlsx',sheet_name='Cuadro 1',header=9)[0:16].T
df.columns = df.iloc[0]
df = df.iloc[:,[0,1,-1]].reset_index(drop=True)[1:]
df.tail(1)


df['ano'] = df.Concepto.ffill().astype(str).str.replace('pr','').str.replace('p','')
df['Colombia'] = round(pd.to_numeric(df.loc[:,df.columns[-2]]),2)
df.index = df.ano.astype(str) + '-' + df.iloc[:,1].astype(str) 

imaela = imaela.merge(df['Colombia'],how='left',left_index=True,right_index=True)



In [115]:
url2 = 'https://www.dane.gov.co' + links[0] # si esta el nuevo mes restarle 1 al indice de links
time.sleep(np.random.randint(0,4,1) + np.random.random())
urllib2.urlretrieve(url2, "colombia.xlsx")

('colombia.xlsx', <http.client.HTTPMessage at 0x7f3899e04c90>)

In [121]:
imaela.merge(df['Colombia'],how='left',left_index=True,right_index=True)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile,Brasil,México,Perú,Ecuador,Bolivia,Paraguay,Colombia
2017-Enero,96.70,101.41,115.421198,219.37,159.54,150.29,305.84,136.632659,NaN,128.56,108.42,151.066765,162.432342,268.331753,106.924102,96.439614
2017-Febrero,96.95,98.97,114.298642,221.50,148.99,154.05,307.27,132.158516,NaN,129.47,105.28,148.057400,141.786469,253.884120,109.528651,96.093843
2017-Marzo,100.84,108.44,118.067834,233.93,153.25,163.25,344.01,152.620959,NaN,141.98,111.90,156.151787,157.139249,290.710792,119.931815,99.858911
2017-Abril,94.83,101.40,114.697920,218.03,143.63,161.07,309.60,151.946345,NaN,133.85,106.20,157.646611,150.920516,305.315250,105.779612,97.065234
2017-Mayo,99.05,110.85,113.723539,225.53,159.80,163.15,316.06,168.389209,NaN,136.11,112.30,164.323037,157.812711,299.148215,108.540354,103.084681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-Diciembre,120.64,123.45,140.752165,279.03,180.08,212.34,396.97,145.823877,120.318627,142.07,112.33,195.058393,176.012995,340.382222,140.144044,128.990318
2022-Enero,106.97,108.41,134.932148,248.73,168.58,186.90,341.51,136.199452,104.597142,132.27,107.32,166.075957,167.160031,NaN,114.314550,110.357840
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.631220,135.52,105.28,163.628001,151.053282,NaN,115.403306,111.837078
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751,NaN,111.96,171.874742,167.054122,NaN,123.845187,115.611480


##### PDF

df = np.array(tb.read_pdf('colombia.pdf',pages = '17',stream=True,multiple_tables=True,pandas_options={'header':None}))
dco = pd.DataFrame(df.reshape(-1,4))
dco = dco.loc[:,0:2]
dco.tail(1)

dco[3] = dco[2].str.split(' ',expand=True)[0] # Tomando las tasas de crecimiento de la tabla pdf
dco.rename(columns={1:'Colombia'},inplace=True)
dco['Colombia1'] = pd.to_numeric(dco[3].str.replace(',','.'),errors='coerce')  #Convirtiendo la data a numero
dco['Colombia'] = pd.to_numeric(dco['Colombia'].str.replace(',','.'),errors='coerce')

split = dco[0].str.split(' - ')     # extraer mes 
fech = []
for i in range(len(split)):
  if type(split[i]) == float:
    fech.append('0')
  elif len(split[i]) == 1 :
    fech.append(str(split[i][0]))
  else:
    fech.append(str(split[i][1].capitalize()))

anof=[]                          # extraer año
for i in range(len(split)):
  if type(split[i]) == float:
    anof.append('0')
  elif len(split[i]) > 1 :
    anof.append(str(split[i][0]))
  else:
    anof.append(np.nan)

dco['mes'] = fech
dco['ano'] = anof
dco['ano'] = dco.ano.replace(['p','r',' '],['','',''],regex=True).ffill()
dco.index = dco['ano'] + '-' + dco['mes'] # creamos indice

## imae


In [73]:
# imaela.drop('a',axis=1,inplace=True)
imaela.drop('a',axis=1,inplace=True)

In [74]:
imaela.tail(3)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Chile,Brasil,México,Perú,Ecuador,Bolivia,Paraguay
2022-Febrero,108.55,109.35,134.593938,248.74,154.54,196.08,365.41,137.408300,98.631220,135.52,105.28,163.628001,151.053282,NaN,115.403306
2022-Marzo,118.00,119.08,139.630203,267.13,158.90,195.02,400.45,153.350828,114.688751,NaN,111.96,171.874742,167.054122,NaN,123.845187
2022-Abril,107.79,NaN,135.895941,250.38,NaN,193.18,NaN,NaN,109.326610,NaN,NaN,171.427214,165.453821,NaN,110.521665


In [93]:
#@title Descarga a Excel
writer = pd.ExcelWriter('imae_latamm.xlsx', engine='xlsxwriter')

#imaela1.to_excel(writer, sheet_name='Interanual')
#imaela2.to_excel(writer, sheet_name='Acumulado')
imaela.to_excel(writer, sheet_name='Indices - Serie Original')
writer.save()

files.download("imae_latamm.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Bar
f, ax = plt.subplots(figsize=(20, 8))
sns.barplot(data=imaela.iloc[-1:,:]);

In [ ]:
#@title Interanual Mes Anterior
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela1.iloc[-2:-1,:]).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Interanual (%) {}'.format(imaela.index[-2]),weight='bold');

In [ ]:
#@title Acumulado Interanual Mes Anterior
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela2.iloc[-2:-1,:]).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Acumulado Interanual (%) a {}'.format(imaela.index[-2]) ,weight='bold');

In [ ]:
#@title Interanual Ultimo Mes
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela1.iloc[-1:,:].dropna(axis=1)).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Interanual (%) {}'.format(imaela.index[-1]),weight='bold')
;

In [ ]:
#@title Acumulado Interanual Ultimo Mes
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela2.iloc[-1:,:].dropna(axis=1)).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Acumulado Interanual (%) a {}'.format(imaela.index[-1]) ,weight='bold');